# Installing and Importing libraries

In [2]:
!pip install kaggle

     ---------------------------------------- 0.0/82.7 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.7 kB ? eta -:--:--
     ------------------ ------------------- 41.0/82.7 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 82.7/82.7 kB 928.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105795 sha256=7fb88a5880c1cc60c625ec0e889ce2f1fd6508ebb03a2d8e0513ac35df944a34
  Stored in directory: c:\users\yps12\appdata\local\pip\cache\wheels\46\d2\26\84d0a1acdb9c6baccf7d28cf06962ec80529fe1ad938489983
Successfully built kaggle


In [3]:
import kaggle

## Download dataset using kaggle api

In [8]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|##########| 200k/200k [00:01<00:00, 205kB/s]
100%|##########| 200k/200k [00:01<00:00, 205kB/s]


## Extract csv from zip file

In [11]:
import zipfile
zip_ref=zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall() #extract file to current directory only
zip_ref.close()

## Read data and handle null values

In [16]:
import pandas as pd
df=pd.read_csv('orders.csv',na_values=['Not Available','unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

## Rename column names: make them lower case and replace with underscore


In [29]:
#method 1
#df.rename(columns={'Order Id':'order_id','City':'city'})

#method 2
# df.columns=df.columns.str.lower()
# df.columns=df.columns.str.replace(' ','_')
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

## Derive new columns discount, sale price and profit

In [34]:
df['discount']=df['list_price']*df['discount_percent']*.01
df['sale_price']=df['list_price']-df['discount']
df['profit']=df['sale_price']-df['cost_price']
df.head(10)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0
5,6,2022-03-13,NaN,Consumer,United States,Los Angeles,California,90032,West,Furniture,Furnishings,FUR-FU-10001487,50,50,7,3,1.5,48.5,-1.5
6,7,2022-12-28,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Art,OFF-AR-10002833,10,10,4,3,0.3,9.7,-0.3
7,8,2022-01-25,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Technology,Phones,TEC-PH-10002275,860,910,6,5,45.5,864.5,4.5
8,9,2023-03-23,NaN,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Binders,OFF-BI-10003910,20,20,3,2,0.4,19.6,-0.4
9,10,2023-05-16,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Appliances,OFF-AP-10002892,90,110,5,3,3.3,106.7,16.7


## Convert order_date from object data type to datetime

In [39]:
df['order_date']= pd.to_datetime(df['order_date'],format="%Y-%m-%d")

## Drop cost price, list_price and discount_percent columns

In [42]:
df.drop(columns=['list_price','cost_price','discount_percent'],inplace=True)

## Connect to SQL server 

In [46]:
import sqlalchemy as sal
engine=sal.create_engine('mssql://Yashesh/Personal Portfolio?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn=engine.connect()

## Load data into SQL server

In [50]:
#method 1 using replace option
df.to_sql('df_orders',con=conn,index=False,if_exists='replace')

38

In [54]:
#method 2 using append option
df.to_sql('df_orders',con=conn,index=False,if_exists='append')

38